# Look at the data

In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from IPython import display

In [2]:
TRAIN = "/data/fashion/img/"

In [3]:
train_list = os.listdir(TRAIN)

In [4]:
dtdf = pd.read_csv("/data/fashion/train.csv")

In [5]:
dtdf

,imageId,url,label
0,1,https://contestimg.wish.com/api/webimage/570f3...,"['95', '66', '137', '70', '20']"
1,2,https://contestimg.wish.com/api/webimage/5468f...,"['36', '66', '44', '214', '105', '133']"
2,3,https://contestimg.wish.com/api/webimage/54641...,"['170', '66', '97', '153', '105', '138']"
3,4,https://contestimg.wish.com/api/webimage/550b9...,"['18', '66', '44', '153', '164']"
4,5,https://contestimg.wish.com/api/webimage/54451...,"['189', '66', '176', '137', '153', '74', '181'..."
5,6,https://contestimg.wish.com/api/webimage/571e0...,"['190', '106', '53', '153', '164', '184']"
6,7,https://contestimg.wish.com/api/webimage/52cbe...,"['114', '222', '113', '176', '214', '87']"
7,8,https://contestimg.wish.com/api/webimage/53994...,"['18', '17', '66', '44', '98', '49', '110', '1..."
8,9,https://contestimg.wish.com/api/webimage/56717...,"['66', '97', '153', '181', '105', '218']"
9,10,https://contestimg.wish.com/api/webimage/562e4...,"['91', '66', '59', '49', '151', '105', '225']"


In [6]:
catelist = ("_".join(list(dtdf["label"].apply(lambda x:"_".join(eval(x)))))).split("_")

In [7]:
cates = list(set(int(i) for i in catelist))

In [8]:
len(cates)

228

In [9]:
from multiprocessing import Pool
from tqdm import trange

In [10]:
CATE = 2

class pick_cate:
    def __init__(self,dtdf,cates,catedir = "/data/fashion/sample/"):
        self.cates = cates
        self.catedir = catedir
        self.dtdf = dtdf
        self.data_list = dtdf.sample(10000).as_matrix().tolist()
    
    def buildcate(self,cate):
        os.system("mkdir -p %s%s"%(self.catedir,cate))
        self.CATE = cate
        data_list = self.data_list
        
        p=Pool(4)
        fnames, isin  = list(zip(*p.map(self.getname,data_list)))
        imglist = np.array(fnames)[np.array(isin)][:20].tolist()
        # print("image list built for cate",str(cate))
        return p.map(self.cpfile, imglist)
        
        # print("image copied for cate",str(cate))
    def getname(self,element):
        iid,url,lbl = element
        fn = "/data/fashion/img/train/%s"%(str(iid)+"p_c"+"_c".join(eval(lbl))+".jpg")
        return fn, (str(self.CATE) in eval(lbl))
    
    def cpfile(self,url):
        try:
            os.system("cp %s %s"%(url, self.catedir+str(self.CATE)))
            return 1
        except:
            return 0
        
    def itercate(self):
        t = trange(len(self.cates))
        for i in t:
            result = self.buildcate(self.cates[i])
            t.set_description("cate:\t%s %s pics"%(self.cates[i],sum(result)))

In [11]:
pick_ = pick_cate(dtdf,cates)

In [12]:
pick_.itercate()

cate:	69 20 pics:  30%|███       | 69/228 [31:00<1:11:26, 26.96s/it]

MemoryError: 